In [1]:
%%time

import os
import re
import string
import pprint

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize

import gensim

Wall time: 19.1 s


In [2]:
%%time

# change working directory

print('Before chdir:', os.getcwd())

os.chdir('C:\\Users\\ravigv\\Documents\\cosine_sim\\cosimV3') 
         
print('After chdir:', os.getcwd())

Before chdir: C:\Users\ravigv
After chdir: C:\Users\ravigv\Documents\cosine_sim\cosimV3
Wall time: 997 µs


In [3]:
%%time
df=pd.read_excel("wb_causecode_asgrp.xlsx")
print("before", df.shape)
df = df.dropna()
print("After", df.shape)

df['combi_feat'] = df['summary']+' '+df['causecode_lv1']+' '+df['causecode_lv2']+' '+df['causecode_lv3']
df['a_cclvl123'] =   df['causecode_lv1']+' '+df['causecode_lv2']+' '+df['causecode_lv3']
df.head()

before (73647, 8)
After (73493, 8)
Wall time: 7.33 s


,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123
1,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0246906,FRB Reconciliation need to be perform on 06/04...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
4,wb,System,WBEI.WBCAS Adobe LiveCycle,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
5,wb,SAP BUSINESS OBJECTS,WBEI.BO.Admin,Request,Enquiry,Information,RTASK0300853,Report Migration From DEV to QA,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information
6,wb,EI (Electronic Invoice),WBEI.SC.EI,Access,Request,Unlock,RTASK0341560,Disable EI Batch Jobs,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock


In [21]:
test1 = df.head(10)

train1 = df.drop(test1.index)
train1 = train1.reset_index(drop = True)

test1 = test1.reset_index(drop = True)


print("train1", train1.shape)
print("test1", test1.shape)

train1 (73483, 10)
test1 (10, 10)


In [22]:
train1.head()

,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123
0,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI.BI - Planning,Access,Issue,Unable to access environment,RTASK0451338,Add an approver to hierarchy,Add an approver to hierarchy Access Issue Unab...,Access Issue Unable to access environment
1,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI FCC GPP - Capital Budgeting System,Data,Request,Quantity Change,RTASK0465145,Change order in CBS,Change order in CBS Data Request Quantity Change,Data Request Quantity Change
2,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI.BI - Planning,Access,Issue,Unable to access environment,RTASK0498800,Substitute approver,Substitute approver Access Issue Unable to acc...,Access Issue Unable to access environment
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Request,Unlock,RTASK0512824,Need to configure drivers in order to refresh...,Need to configure drivers in order to refresh...,Access Request Unlock
4,wb,ADOBE LIVECYCLE EFORMS,WBEI.WBCAS Adobe LiveCycle,Access,Request,Unlock,RTASK0515282,[Urgent] Assistance with filling out eform55,[Urgent] Assistance with filling out eform55 A...,Access Request Unlock


In [23]:
test1.head()

,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123
0,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement
1,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0246906,FRB Reconciliation need to be perform on 06/04...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
2,wb,System,WBEI.WBCAS Adobe LiveCycle,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
3,wb,SAP BUSINESS OBJECTS,WBEI.BO.Admin,Request,Enquiry,Information,RTASK0300853,Report Migration From DEV to QA,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information
4,wb,EI (Electronic Invoice),WBEI.SC.EI,Access,Request,Unlock,RTASK0341560,Disable EI Batch Jobs,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock


In [24]:
%%time
##############################################################################################################################
# Function to keep required columns in a dataframe

def keepcols(df):   
    return df[['causecode_lv1', 'causecode_lv2', 'causecode_lv3', 'summary', 'incident_id']]
##############################################################################################################################
# Checking and imputing missing values

def check_replace_missing(df, na_fill):
#     print("Before imputing missing values", df1.isnull().values.any())
    df = df.fillna(na_fill)
    df = df.dropna()
#     print("After imputing missing values", df.isnull().values.any())
    return df
##############################################################################################################################
# Preprocess text 

import re
from gensim.parsing.preprocessing import remove_stopwords

def clean_text(sentence, stopwords=True):
    
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #lower case 
    sentence = sentence.lower()
    # square brackets
    sentence = re.sub('\[.*?\]', '', sentence)  
    # hyperlinks
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    # remove punctuation
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    # remove year
    sentence = re.sub(r"\b(19[40][0-9]|20[0-1][0-9]|2020)\b",'',sentence)
    # remove month names
    mp = r"(\b\d{1,2}\D{0,3})?\b(?:Jan|jan(?:uary)?|Feb|feb(?:ruary)?|Mar|mar(?:ch)?|Apr|mar(?:il)?|May|may|Jun|jun(?:e)?|Jul|jul(?:y)?|Aug|aug(?:ust)?|Sep|sep(?:tember)?|Oct|oct(?:ober)?|(Nov|nov|Dec|dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*"
    sentence = re.sub(mp, '', sentence)
     
    # remove words containing numbers
    sentence = re.sub('\w*\d\w*', '', sentence)
    # remove numbers 
    sentence = re.sub(r'[0-9]+','',sentence)
    
    if stopwords:
        sentence = remove_stopwords(sentence)
    
    return sentence
##############################################################################################################################
# Get clean sentences with numpy

def get_cleaned_sents(df, col_name, stopwords=False):
   # find the index no 
    index_no = df.columns.get_loc(col_name)
    # Convert to numpy array
    df_np = df.to_numpy()
    
    rows = [clean_sentence(row[index_no]) for row in df_np]
    return rows
##############################################################################################################################
# Sentence or text tokenization

# from nltk.corpus import stopwords
# from stop_words import get_stop_words
# from nltk.tokenize import word_tokenize


# stop_words = list(get_stop_words('en'))  # About 900 stopwords
# nltk_words = list(stopwords.words('english'))  # About 150 stopwords
# stop_words.extend(nltk_words)  # Combining both of the stop word list
# stop_words.extend(['system','sap','issue', 'user', 'unable', 'Unable', 'User'])

# def tokenization(sentence):
#     token_words = word_tokenize(sentence)
#     token_words = [word for word in token_words if word not in stop_words]  
#     return " ".join(token_words)
##############################################################################################################################
# Get cosine similarity   

import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity(x_set, y_set):

    #     print(w1)
    #     print(w2)
    #     x_set = word_tokenize(w1)
    #     y_set = word_tokenize(w2)

    # sw contains the list of stopwords
    #sw = stopwords.words('english')
    l1 = []
    l2 = []

    # remove stop words from string
    #x_set = {w for w in X_list if not w in sw}
    #y_set = {w for w in Y_list if not w in sw}

    x_set = set(x_set)
    y_set = set(y_set)

    # form a set containing keywords of both strings
    rvector = set(x_set).union(set(y_set))
    for w in rvector:
        if w in x_set:
            l1.append(1)  # create a vector
        else:
            l1.append(0)
        if w in y_set:
            l2.append(1)
        else:
            l2.append(0)
    c = 0

    # cosine formula
    for z in range(len(rvector)):
        c += l1[z] * l2[z]
    cosine = c / float((sum(l1) * sum(l2))**0.5)
    #print("similarity: ", cosine)
    return cosine
##############################################################################################################################
# Get jaccard similarity   

def jaccard_similarity_mod(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1)


def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

Wall time: 0 ns


In [25]:
%%time
train1["clean_txt"] = train1["combi_feat"].apply(clean_text)
test1["clean_txt"] = test1["combi_feat"].apply(clean_text)

Wall time: 3.1 s


In [26]:
%%time
train1.head()

Wall time: 998 µs


,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123,clean_txt
0,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI.BI - Planning,Access,Issue,Unable to access environment,RTASK0451338,Add an approver to hierarchy,Add an approver to hierarchy Access Issue Unab...,Access Issue Unable to access environment,add approver hierarchy access issue unable acc...
1,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI FCC GPP - Capital Budgeting System,Data,Request,Quantity Change,RTASK0465145,Change order in CBS,Change order in CBS Data Request Quantity Change,Data Request Quantity Change,change order cbs data request quantity change
2,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI.BI - Planning,Access,Issue,Unable to access environment,RTASK0498800,Substitute approver,Substitute approver Access Issue Unable to acc...,Access Issue Unable to access environment,substitute approver access issue unable access...
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Request,Unlock,RTASK0512824,Need to configure drivers in order to refresh...,Need to configure drivers in order to refresh...,Access Request Unlock,need configure drivers order refresh lumira bi...
4,wb,ADOBE LIVECYCLE EFORMS,WBEI.WBCAS Adobe LiveCycle,Access,Request,Unlock,RTASK0515282,[Urgent] Assistance with filling out eform55,[Urgent] Assistance with filling out eform55 A...,Access Request Unlock,assistance filling access request unlock


In [27]:
test1.head()

,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123,clean_txt
0,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement,deal status payment progress payment delete da...
1,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0246906,FRB Reconciliation need to be perform on 06/04...,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...,frb reconciliation need perform access approva...
2,wb,System,WBEI.WBCAS Adobe LiveCycle,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...,urgent forward eform approver access approval ...
3,wb,SAP BUSINESS OBJECTS,WBEI.BO.Admin,Request,Enquiry,Information,RTASK0300853,Report Migration From DEV to QA,Report Migration From DEV to QA Request Enquir...,Request Enquiry Information,report migration dev qa request enquiry inform...
4,wb,EI (Electronic Invoice),WBEI.SC.EI,Access,Request,Unlock,RTASK0341560,Disable EI Batch Jobs,Disable EI Batch Jobs Access Request Unlock,Access Request Unlock,disable ei batch jobs access request unlock


In [28]:
%%time
train1['inc_bigram_tokens'] = train1['clean_txt'].apply(lambda x: [ ' '.join((a, b)) 
                           for a, b in nltk.bigrams(nltk.word_tokenize(x))])

train1['inc_trigram_tokens'] = train1['clean_txt'].apply(lambda x: [' '.join((a, b, c)) 
                            for a, b, c in nltk.trigrams( nltk.word_tokenize(x))])

train1['inc_single_tokens'] = train1['clean_txt'].apply( lambda x: word_tokenize(x))
        

Wall time: 23.7 s


In [29]:
%%time
train1.head()

Wall time: 0 ns


,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123,clean_txt,inc_bigram_tokens,inc_trigram_tokens,inc_single_tokens
0,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI.BI - Planning,Access,Issue,Unable to access environment,RTASK0451338,Add an approver to hierarchy,Add an approver to hierarchy Access Issue Unab...,Access Issue Unable to access environment,add approver hierarchy access issue unable acc...,"[add approver, approver hierarchy, hierarchy a...","[add approver hierarchy, approver hierarchy ac...","[add, approver, hierarchy, access, issue, unab..."
1,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI FCC GPP - Capital Budgeting System,Data,Request,Quantity Change,RTASK0465145,Change order in CBS,Change order in CBS Data Request Quantity Change,Data Request Quantity Change,change order cbs data request quantity change,"[change order, order cbs, cbs data, data reque...","[change order cbs, order cbs data, cbs data re...","[change, order, cbs, data, request, quantity, ..."
2,wb,SAP - CBS (CAPITAL BUDGET SYSTEM),WBEI.BI - Planning,Access,Issue,Unable to access environment,RTASK0498800,Substitute approver,Substitute approver Access Issue Unable to acc...,Access Issue Unable to access environment,substitute approver access issue unable access...,"[substitute approver, approver access, access ...","[substitute approver access, approver access i...","[substitute, approver, access, issue, unable, ..."
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Request,Unlock,RTASK0512824,Need to configure drivers in order to refresh...,Need to configure drivers in order to refresh...,Access Request Unlock,need configure drivers order refresh lumira bi...,"[need configure, configure drivers, drivers or...","[need configure drivers, configure drivers ord...","[need, configure, drivers, order, refresh, lum..."
4,wb,ADOBE LIVECYCLE EFORMS,WBEI.WBCAS Adobe LiveCycle,Access,Request,Unlock,RTASK0515282,[Urgent] Assistance with filling out eform55,[Urgent] Assistance with filling out eform55 A...,Access Request Unlock,assistance filling access request unlock,"[assistance filling, filling access, access re...","[assistance filling access, filling access req...","[assistance, filling, access, request, unlock]"


In [30]:
# %%time
# query= "PLM health check and batch job monitoring for the week 01-01-2018 to 05-01-2018"

# main_bigrams = tuple([' '.join((a, b)) for a, b in nltk.bigrams(nltk.word_tokenize(query))])

# main_trigrams = tuple([' '.join((a, b,c)) for a, b,c in nltk.trigrams(nltk.word_tokenize(query))])
# # print('main bigrams are:', main_bigrams)

# main_tokens= tuple(nltk.word_tokenize(query))

# main_tokens

In [34]:
test1['inc_bigram_tokens'] = test1['clean_txt'].apply(lambda x: [ ' '.join((a, b)) 
                           for a, b in nltk.bigrams(nltk.word_tokenize(x))])

test1['inc_trigram_tokens'] = test1['clean_txt'].apply(lambda x: [' '.join((a, b, c)) 
                            for a, b, c in nltk.trigrams( nltk.word_tokenize(x))])

test1['inc_single_tokens'] = test1['clean_txt'].apply( lambda x: word_tokenize(x))
test1.head(1)       

,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary,combi_feat,a_cclvl123,clean_txt,inc_bigram_tokens,inc_trigram_tokens,inc_single_tokens
0,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i...","Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement,deal status payment progress payment delete da...,"[deal status, status payment, payment progress...","[deal status payment, status payment progress,...","[deal, status, payment, progress, payment, del..."


In [35]:
 test1.loc[test1.incident_id == similar_df.at[0, 'incident_id'], 'inc_single_tokens'])[0]

SyntaxError: invalid syntax (<ipython-input-35-e8469a48590e>, line 1)

In [ ]:
%%time
df_inc_similarity_score = pd.DataFrame()

for i in test1.index:
    print(i)
    
    main_tokens = []
    main_bigrams = []
    main_trigrams = []
#     query = test1.at[i,'clean_txt']
#     query = clean_text(query, True)
    main_tokens = tuple(test1.at[i, 'inc_single_tokens'])
    main_bigrams = tuple(test1.at[i, 'inc_bigram_tokens'])
    main_trigrams= tuple(test1.at[i, 'inc_trigram_tokens'])
    
#     most_similar_inc_list = {}
    for incident in train1.index:
        if incident == '':
            continue
        compare_token = []
        compare_bigram = []
        compare_trigram = []
        # print(incident)
        compare_token = tuple(train1.at[incident, 'inc_single_tokens'])
        compare_bigram = tuple(train1.at[incident, 'inc_bigram_tokens'])
        compare_trigram = tuple(train1.at[incident, 'inc_trigram_tokens'])


        #print("this is compare description - {0}".format(compare_token))
        #print("this is compare description - {0}".format(compare_bigram))
        #print("this is compare description - {0}".format(compare_trigram))
        #print("length of main desc  is {0}".format(len(main_tokens)))
        #print("length of compare desc  is {0}".format(len(compare_desc)))
        count = 0
        score = 0
        if len(compare_token) == 0 or len(main_tokens) == 0:
            sim_token = 0
        else:
            sim_token = jaccard_similarity_mod(main_tokens, compare_token)
        if len(compare_bigram) == 0 or len(main_bigrams) == 0:
            sim_bigram = 0
        else:
            sim_bigram = cosine_similarity(main_bigrams, compare_bigram)
        if len(main_trigrams) == 0 or len(compare_trigram) == 0:
            sim_trigram = 0
        else:
            sim_trigram = cosine_similarity(main_trigrams, compare_trigram)
        #print("this is similarity - {0}".format(sim))
        if sim_token > 0:
            count = count + 1
        if sim_bigram > 0:
            count = count + 1
        if sim_trigram > 0:
            count = count + 1
            # +'-'+str(round(sim,2)))
        if count > 0:
            score = (sim_token + sim_bigram + sim_trigram) / count
#             most_similar_inc_list[incident] = score
            df_inc_similarity_score = df_inc_similarity_score.append(
                {
#                     'incident_id': train1.at[incident,'incident_id'],
#                     'most_similar_incident': test1.at[i,'incident_id'],
                    'incident_id': test1.at[i,'incident_id'],
                    'most_similar_incident': train1.at[incident,'incident_id'],
                    'tokens_score': sim_token,
                    'bigrams_score': sim_bigram,
                    'trigrams_score': sim_trigram,
                    'inc_similarity_score': score},
                ignore_index=True)
            #print(datetime.now())

0
1
2
3
4
5
6
7


In [47]:
print(df_inc_similarity_score.shape)
df_inc_similarity_score

(213981, 6)


,bigrams_score,inc_similarity_score,incident_id,most_similar_incident,tokens_score,trigrams_score
0,0.000000,0.111111,RTASK0086491,RTASK0465145,0.111111,0.00000
1,0.000000,0.111111,RTASK0086491,RTASK0668910,0.111111,0.00000
2,0.000000,0.111111,RTASK0086491,RTASK0694905,0.111111,0.00000
3,0.000000,0.111111,RTASK0086491,RTASK0851158,0.111111,0.00000
4,0.000000,0.111111,RTASK0086491,RTASK0944546,0.111111,0.00000
...,...,...,...,...,...,...
213976,0.534522,0.554700,RTASK0361872,RTASK1758411,0.666667,0.46291
213977,0.000000,0.166667,RTASK0361872,RTASK1758619,0.166667,0.00000
213978,0.142857,0.238095,RTASK0361872,RTASK1758679,0.333333,0.00000
213979,0.142857,0.238095,RTASK0361872,RTASK1758684,0.333333,0.00000


In [ ]:
%%time

df_inc_similarity_score = df_inc_similarity_score.sort_values(['incident_id','inc_similarity_score'],ascending=False).groupby('incident_id').head(5).reset_index(drop=True)


In [20]:
%%time

df_inc_similarity_score = df_inc_similarity_score.sort_values('inc_similarity_score', ascending  = False).head(5)
df_inc_similarity_score

Wall time: 52.1 ms


,bigrams_score,inc_similarity_score,incident_id,most_similar_incident,tokens_score,trigrams_score
10193,0.544331,0.524743,RTASK0086491,RTASK2055237,0.555556,0.474342
10699,0.544331,0.524743,RTASK0086491,RTASK2130731,0.555556,0.474342
257,0.503953,0.497507,RTASK0086491,RTASK1196843,0.555556,0.433013
1372,0.503953,0.497507,RTASK0086491,RTASK1275565,0.555556,0.433013
353,0.503953,0.497507,RTASK0086491,RTASK1203876,0.555556,0.433013


In [ ]:
df_inc_similarity_score.to_excel("df_inc_similarity_score_V1.xlsx")

In [41]:
qids = []
tids = []
sim_st = []
for i in test1.index:
    query = test1.at[i, 'inc_single_tokens']
    print(query)
    
    
    for j in train1.index:
        target = train1.at[j, 'inc_single_tokens']
        
        sim_token = jaccard_similarity_mod(query, target)
        sim_bigram = cosine_similarity(main_bigrams, compare_bigram)
        
        sim_st.append(sim_token)
        qids.append(test1.at[i, 'incident_id'])
        tids.append(train1.at[j, 'incident_id'])

dff = pd.DataFrame(list(zip(qids, tids, sim_st)),
               columns =['qids', 'tids', 'sim_st'])
dff

['deal', 'status', 'payment', 'progress', 'payment', 'delete', 'data', 'deletion', 'user', 'requirement']
['frb', 'reconciliation', 'need', 'perform', 'access', 'approval', 'accesssap', 'issueunable', 'access', 'environment']
['urgent', 'forward', 'eform', 'approver', 'access', 'approval', 'accesssap', 'issueunable', 'access', 'environment']
['report', 'migration', 'dev', 'qa', 'request', 'enquiry', 'information']
['disable', 'ei', 'batch', 'jobs', 'access', 'request', 'unlock']
['complete', 'request', 'general', 'service', 'request', 'access', 'request', 'unlock']
['hpqc', 'access', 'project', 'workday', 'domain', 'mis', 'id', 'xvdhake', 'access', 'request', 'remove', 'access']
['help', 'withsetting', 'cbs', 'profile', 'wbconnect', 'access', 'issue', 'unable', 'access', 'environment']
['route', 'ticket', 'new', 'approver', 'access', 'request', 'unlock']
['proactive', 'breakfix', 'incident', 'ticket', 'monthly', 'release', 'production', 'access', 'request', 'unlock']


,qids,tids,sim_st
0,RTASK0086491,RTASK0451338,0.000000
1,RTASK0086491,RTASK0465145,0.111111
2,RTASK0086491,RTASK0498800,0.000000
3,RTASK0086491,RTASK0512824,0.000000
4,RTASK0086491,RTASK0515282,0.000000
...,...,...,...
734825,RTASK0437285,WBTINC0209848,0.200000
734826,RTASK0437285,WBTINC0209849,0.200000
734827,RTASK0437285,WBTINC0209896,0.100000
734828,RTASK0437285,WBTINC0209907,0.100000


In [48]:
qids = []
tids = []
sim_st = []
sim_bi = []
sim_tri = []


for i in test1.index:     
    
    for j in train1.index:
        
        sim_token = jaccard_similarity_mod(test1.at[i, 'inc_single_tokens'], train1.at[j, 'inc_single_tokens'])
        sim_bigram = cosine_similarity(test1.at[i, 'inc_bigram_tokens'], train1.at[j, 'inc_bigram_tokens'])
        sim_trigram = cosine_similarity(test1.at[i, 'inc_trigram_tokens'], train1.at[j, 'inc_trigram_tokens'])
        
        
        
        qids.append(test1.at[i, 'incident_id'])
        tids.append(train1.at[j, 'incident_id'])
        sim_st.append(sim_token)
        sim_bi.append(sim_bigram)
        sim_tri.append(sim_trigram)
        

        
dff = pd.DataFrame(list(zip(qids, tids, sim_st, sim_bi, sim_tri)),
               columns =['test_incid', 'train_incid', 'sim_token','sim_bigram','sim_trigram'])
dff

ZeroDivisionError: float division by zero